In [1]:
import pandas as pd
import os

with open('data/bpc_blogs.json', 'r') as f:
    articles = pd.read_json(f)

md_path = "data/markdown_articles/"

md_paths = []
for hash in articles['hash_id']:
    file_path = os.path.join(md_path, f"{hash}.md")
    md_paths.append(file_path)

In [6]:
# print(f'length of md_paths: {len(md_paths)}')

with open(md_paths[0], 'r', encoding='utf-8') as f:
    content = f.read()

content

'---\ntitle: "Fix Our Forests Act: A Bipartisan Breakthrough for America’s Forests"\ndate: "Oct 22, 2025"\nurl: "https://bipartisanpolicy.org/blog/fix-our-forests-act-a-bipartisan-breakthrough-for-americas-forests/"\ntype: "Blog Post"\ntags: ["Energy Policy"]\npolicy_areas: ["Energy"]\nrelated_people: ["Sean Babington", "Mariam Al-Shamma"]\n---\n\nOn October 21, 2025, the U.S. Senate Committee on Agriculture, Nutrition, and Forestry voted 18-5 to advance the [Fix Our Forests Act (FOFA)](https://www.congress.gov/bill/119th-congress/senate-bill/1462/text). This strong bipartisan vote on comprehensive forestry legislation—shepherded by Agriculture Committee Chairman John Boozman (R-AK) and the committee’s ranking member, Senator Amy Klobuchar (D-MN)—marks yet another milestone in policymakers’ renewed focus on consensus-driven reforms to govern America’s forested lands. FOFA _,_ a version of which has already passed the House with support from both parties, provides a host of new authorit